In [ ]:
import pandas as pd
import numpy as np
from xyt import FakeDataGenerator, GPSDataProcessor, GPSDataPrivacy
from xyt import plot_gps_on_map
from privacy import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## REMAINS TO DO : 

GPSDataProcessor --> FIX THE MULTIPROCESSING AND TEST

In [ ]:
fakegps = FakeDataGenerator(location_name="Suisse", num_users=5, home_radius_km = 20)
waypoints = fakegps.generate_waypoints(num_rows=12, num_extra_od_points=10, max_displacement_meters = 10)
legs = fakegps.generate_legs(num_rows=12)
stays = fakegps.generate_staypoints(num_rows=12)

In [ ]:
waypoints.columns

In [ ]:
data_processor = GPSDataProcessor(radius=0.03)
poi_waypoints = data_processor.guess_home_work(waypoints, cell_size=0.3)
smoothed_df = data_processor.smooth(poi_waypoints, sigma=10)
segmented_df = data_processor.segment(smoothed_df)
mode_df = data_processor.mode_detection(segmented_df)
legs_ = data_processor.get_legs(df = mode_df)

In [ ]:
poi_waypoints

In [ ]:
plot_gps_on_map(poi_waypoints, home_col='home_loc', work_col='work_loc')

## NEXT: ADD THE INSTANCE PRIVACY

You need to get a list of unique tuples of areas you want to obfuscate first

In [ ]:
df = poi_waypoints.copy()

# Create a list of unique tuples from 'home_loc' and 'work_loc' columns
unique_tuples = list(set((point.x, point.y) for point in df['home_loc']))
unique_tuples.extend(set((point.x, point.y) for point in df['work_loc']))
unique_tuples = list(set(unique_tuples))

print(unique_tuples)

In [ ]:
GPSDataPrivacy = GPSDataPrivacy()
df, loc = GPSDataPrivacy.obfuscate(poi_waypoints, unique_tuples, radius=2000)

In [ ]:
loc[0]

It would be nice to have a circle around the obfuscated places

In [ ]:
plot_gps_on_map(df, home_col='home_loc', work_col='work_loc')

In [ ]:
import folium
from folium import plugins  # Import the 'plugins' module


aggregated_waypoints = GPSDataPrivacy.aggregate(poi_waypoints).reset_index()
possible_times = aggregated_waypoints.tracked_at.dt.date.unique()

#save selection
selected_timestamp = possible_times[-1]  # You can use a default value or interactively select it
selected_agg_data = aggregated_waypoints[aggregated_waypoints['tracked_at'].dt.date == selected_timestamp]
points = np.column_stack((aggregated_waypoints.cell_longitude.values, aggregated_waypoints.cell_latitude.values))

m = folium.Map(location=points[0])
hm = plugins.HeatMap(points)
m.add_child(hm)

display(m)


In [ ]:
possible_times